# Visual Testing

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def group_contours(contours, horizontal_proximity=30, vertical_proximity=30):
    # Defines a function to determine if two contours are close to each other
    def are_close(c1, c2):
        x1, y1, w1, h1 = cv2.boundingRect(c1)
        x2, y2, w2, h2 = cv2.boundingRect(c2)

        # Check horizontal closeness
        horizontal_close = (x2 <= x1 + w1 + horizontal_proximity) and (x1 <= x2 + w2 + horizontal_proximity)

        # Check vertical closeness
        vertical_close = (y2 <= y1 + h1 + vertical_proximity) and (y1 <= y2 + h2 + vertical_proximity)

        return horizontal_close and vertical_close

    groups = []
    used = set()

    # Iteratively group close contours
    for i in range(len(contours)):
        if i in used:
            continue
        group = [contours[i]]
        queue = [i]
        used.add(i)
        while queue:
            idx = queue.pop(0)
            for j in range(len(contours)):
                if j not in used and are_close(contours[idx], contours[j]):
                    used.add(j)
                    queue.append(j)
                    group.append(contours[j])
        groups.append(group)
    return groups

def draw_bounding_boxes_on_changes_and_save(image_path1, image_path2, save_path):
    # Load images
    img1 = cv2.imread(image_path1)
    img2 = cv2.imread(image_path2)

    if img1 is None or img2 is None:
        raise FileNotFoundError("One or both images could not be loaded.")

    # Ensure images are the same size
    if img1.shape != img2.shape:
        raise ValueError("Images must have the same dimensions.")

    # Convert images to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Compute the difference and apply threshold
    diff = cv2.absdiff(gray1, gray2)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Find contours from the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out very small contours
    filtered_contours = [c for c in contours if cv2.contourArea(c) > 0]
    contour_groups = group_contours(filtered_contours)

    # Draw bounding boxes around contour groups
    for group in contour_groups:
        all_points = np.vstack([cv2.boxPoints(cv2.minAreaRect(contour)) for contour in group])
        x, y, w, h = cv2.boundingRect(all_points)
        cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save and display the processed image
    cv2.imwrite(save_path, img2)

    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    plt.imshow(img2_rgb)
    plt.title('Second Image with Bounding Boxes')
    plt.axis('off')
    plt.show()

# Example usage
draw_bounding_boxes_on_changes_and_save('with_element.png', 'with_out_symbol.png', 'processed_image.jpg')


# Remove Frames

In [ ]:
import cv2
import numpy as np

def remove_stagnant_frames(video_path, output_path, change_threshold=30, area_threshold=100):
    # Setup video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError("Video file could not be opened.")

    # Obtain input video frame rate and resolution
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Setup video writer with the same frame rate as the input video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
 
    # Read the first frame
    ret, frame1 = cap.read()
    if not ret:
        raise ValueError("Cannot read video file.")
    
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

    while True:
        # Read next frame
        ret, frame2 = cap.read()
        if not ret:
            break
        
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate the absolute difference and threshold
        diff = cv2.absdiff(gray1, gray2)
        _, thresh = cv2.threshold(diff, change_threshold, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        significant_change = any(cv2.contourArea(contour) > area_threshold for contour in contours)

        if significant_change:
            out.write(frame2)
            gray1 = gray2  # Update the reference frame to the current frame

    # Cleanup
    cap.release()
    out.release()
    print("Processing complete. Output saved to:", output_path)

# Example usage
remove_stagnant_frames('/Users/jatavathpavannaik/Documents/ICodeTest/Screen Recording 2024-05-19 at 5.25.28 PM.mov', 'output_video.mp4')
